In [ ]:
import os

from DataLoader import (
    config,
    loader
)

import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import curve_fit

import matplotlib.pyplot as plt
import seaborn as sns

from copy import deepcopy
 
from sklearn.preprocessing import (
    MinMaxScaler,
    PolynomialFeatures
)
from sklearn.decomposition import PCA

from sklearn.metrics import mean_squared_error

In [ ]:
data_path = '../data/raw/'

files = os.listdir(data_path)

# make a dict, key - file_name_last part
dta = dict()

for file in files:
    df = pd.read_csv(os.path.join(data_path, file), skiprows=config.COUNT_SKIP, sep=';')
    df = loader.fill_empty(loader.transform_header(df))
    splitted = loader.split(df.columns[1:])
    group = loader.group(splitted, df)

    dta[file] = group

In [ ]:
def exponential_moving_average(data, window):
    weights = np.exp(np.linspace(-1., 0., window))
    weights /= weights.sum()
    ema = np.convolve(data, weights, mode='full')[:len(data)]
    ema[:window] = ema[window]
    return ema

In [ ]:
pzn = dta['month_csv_acceleration_10khz.csv']['Н']['ПЗН'][-2:]
r_acc = 1 / np.sqrt(pzn[0] ** 2 + pzn[1] ** 2)
pzn.append(r_acc)

да всё равно тут нет тренда ростового

In [ ]:
pzn

сжимаем

In [ ]:
pca = PCA(n_components=2)
compressed = pca.fit_transform(pd.DataFrame(pzn).T)

видим, что первая компонента немного отображает трендовую составляющую

In [ ]:
sns.scatterplot(abs(compressed[:, 0]))

определяем функцию деградации